## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [5]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Hit:13 http://pp

In [6]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [7]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [8]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [9]:
corpus[:50]

["UNIST(울산과학기술원)은 번개가 치는 원리를 이용한 '인공 번개 마찰 전기 발전기'를 개발했다고 6일 밝혔다",
 '번개는 구름 속 수증기 분자가 얼음 결정과 마찰하는 과정에서 마이너스 전자가 분리돼 축적됐다가 플러스 전자인 땅으로 떨어지며 순간적으로 엄청난 에너지를 방출하면서 생긴다',
 '백정민 신소재공학부 교수와 미국 조지아공대(Georgia Tech), 성균관대, KIST(한국과학기술연구원), 경희대 공동연구팀은 번개 원리를 극대화할 수 있는 3층짜리 구조의 발전기를 만들었다',
 '마치 구름 속 수증기와 얼음 결정 사이에 새로운 물질을 넣어 마찰이 더 일어나도록 한 것이다',
 '연구팀은 "접지층은 마찰로 생성된 전하가 유실되는 것을 막는 역할도 한다"며 "기존 2층짜리 마찰 전기 발전기보다 16배 이상 출력이 높았다"고 설명했다',
 "이번 연구는 '네이처 커뮤니케이션(Nature Communications)' 온라인판에 실렸다",
 '- 움직임 재현과 함께 공간의 차이 인지',
 '환경에 적응',
 '- 증강현실 아바타, 다양한 역할 수행할 것으로 예측',
 '국내 연구진에 의해 증강현실 아바타의 발판이 될 신기술이 개발됐다',
 'KAIST는 문화기술대학원 이성희 교수 연구팀이 사용자의 움직임을 재현하면서도 원격 공간의 차이를 인지해 바뀐 환경에 적응할 수 있는 증강현실 아바타 기술을 개발했다고 20일 밝혔다',
 '이성희 교수 연구팀은 사용자의 공간에는 식탁용 의자, 원격 공간에는 일인용 소파를 놓은 뒤 사용자가 식탁용 의자에 앉으면 아바타는 소파에 적합한 동작과 자세로 변형해 자리에 앉는 실험에 성공했다',
 '증강현실 아바타는 사용자를 대신해 가상공간 혹은 원격공간에 존재하며, 사용자의 움직임을 그대로 따라 하고 반영한다',
 '그러나 사용자의 공간과 아바타가 존재하는 공간에 차이가 있기 때문에 사용자의 동작을 아바타에 그대로 적용하면 아바타는 가상, 원격 공간의 물체를 그냥 통과해버리는 등 사실성이 떨어지게 된다',
 '연구팀은 이

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

*  'Josa'와 'Punctuation'은 큰 의미를 지니고 있지 않아서 제거하였다.
*  'Number'는 각 기사에서는 중요한 의미를 가질 수는 있지만 앞뒤 문맥 없이 단어들을 추출해냈을 때는 큰 의미를 가지지 않을 것으로 판단되어 제거하였다.
*  형태소와 품사를 추출해주는 작업을 하였다.

In [10]:
okt = Okt()

In [19]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence, stop = ['Josa','Punctuation', 'Number']):
    tokenized = []
    for i, j in okt.pos(sentence, stem = True, norm = True):
        if j in stop:
            continue
        tokenized.append(i)
    return tokenized 

In [20]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [22]:
CustomTokenization(corpus[0])

['UNIST',
 '울산',
 '과학기술원',
 '은',
 '번개',
 '치다',
 '원리',
 '이용',
 '인공',
 '번개',
 '마찰',
 '전기',
 '발전기',
 '를',
 '개발',
 '하다',
 '밝히다']

In [24]:
CustomTokenization(corpus[1])

['번개',
 '구름',
 '속',
 '수증기',
 '분자',
 '얼음',
 '결정',
 '마찰',
 '하다',
 '과정',
 '마이너스',
 '전자',
 '분리',
 '돼다',
 '축적',
 '돼다',
 '플러스',
 '전자',
 '땅',
 '떨어지다',
 '순간',
 '적',
 '엄청나다',
 '에너지',
 '방출',
 '하다',
 '생기다']

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

*  사전학습을 할 때에 size를 200으로 맞추어줄 것이기 때문에 사전학습 없는 모델 또한 size를 200으로 동일하게 설정하여 사전학습의 효과를 더욱 쉽게 파악하도록 하였다.
*  한번이라도 나온 단어는 유의미하다고 판단되어 min count는 0으로 설정하였다.
*  epochs는 충분한 학습을 위해 30으로 설정하였다.

In [25]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count = 0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3869


In [26]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30)

(468758, 529680)

In [27]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['UNIST', '울산', '과학기술원', '은', '번개', '치다', '원리', '이용', '인공', '마찰', '전기', '발전기', '를', '개발', '하다', '밝히다', '구름', '속', '수증기', '분자', '얼음', '결정', '과정', '마이너스', '전자', '분리', '돼다', '축적', '플러스', '땅', '떨어지다', '순간', '적', '엄청나다', '에너지', '방출', '생기다', '백', '정민', '신소재공학', '부', '교수', '미국', '조지아공대', 'Georgia', 'Tech', '성균관대', 'KIST', '한국', '과학기술', '연구원', '경희대', '공', '동', '연', '구', '팀', '극대', '화하다', '수', '있다', '층', '짜다', '구조', '만들다', '마치', '사이', '새롭다', '물질', '넣다', '더', '일어나다', '것', '접', '지층', '생', '성', '되다', '전하', '유실', '막다', '역할', '며', '기존', '배', '이상', '출력', '높다', '고', '설명', '이번', '연구', '네이처', '커뮤니케이션', 'Nature', 'Communications', '온라인', '판', '실리다', '움직임', '재현', '함께', '공간', '차이', '인지', '환경', '적응', '증강현실', '아바타', '다양하다', '수행', '예측', '국내', '구진', '의하다', '발판', '신', '기술', 'KAIST', '늘다', '문화', '대학원', '이성희', '사용자', '원격', '바뀌다', '적응하다', '식탁', '용', '의자', '일', '인용', '소파', '놓다', '뒤', '앉다', '적합하다', '동작', '자세', '변형', '자리', '실험', '성공하다', '대신', '가상공간', '혹은', '원', '격', '존재', '그대로', '따르다', '반영', '다', '그러나', '때문

In [28]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('기상청')

[('보고', 0.9984304904937744),
 ('모양', 0.9984217882156372),
 ('산소', 0.9984002113342285),
 ('대한', 0.9983939528465271),
 ('통계', 0.9983848929405212),
 ('with', 0.9983822107315063),
 ('생기다', 0.9983798265457153),
 ('인하다', 0.9983742237091064),
 ('누구', 0.9983740448951721),
 ('배우다', 0.9983683824539185)]

In [29]:
model_no_pretrained.wv.most_similar('변화')

[('찾다', 0.9997676610946655),
 ('없이', 0.9997662901878357),
 ('생산', 0.999754786491394),
 ('이미지', 0.9997429847717285),
 ('방향', 0.9997165203094482),
 ('시간', 0.999713659286499),
 ('최대', 0.9996935129165649),
 ('단계', 0.9996856451034546),
 ('추적', 0.9996681809425354),
 ('높다', 0.9996644258499146)]

In [31]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3869


In [32]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31473


In [33]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30)

(521819, 529680)

In [34]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('기상청')

[('속보', 0.5775933265686035),
 ('표준시', 0.5563241243362427),
 ('국세청', 0.532078206539154),
 ('관세청', 0.5183442831039429),
 ('기상', 0.5167145133018494),
 ('통계청', 0.513887345790863),
 ('병무청', 0.49296754598617554),
 ('태풍', 0.4916233420372009),
 ('조달청', 0.476013720035553),
 ('산림청', 0.46051180362701416)]

In [35]:
model_pretrained.wv.most_similar('변화')

[('변동', 0.6799405813217163),
 ('가져오다', 0.6076920032501221),
 ('차이', 0.6075598001480103),
 ('흔하다', 0.5984135866165161),
 ('일정하다', 0.5951571464538574),
 ('시키다', 0.5926815271377563),
 ('변천', 0.5914894342422485),
 ('무질서', 0.5817601680755615),
 ('강하다', 0.5808844566345215),
 ('않다', 0.5780043005943298)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

두 모델 모두 같은 hyperparameter를 지정하여 학습을 진행하였는데, 사전학습이 있는 상태에서 학습한 모델이 그렇지 않은 모델에 비해서 훨씬 성능이 높은 것으로 보여진다. 사전 학습이 없던 모델에서는 기상청과 변화라는 단어와 전혀 관련이 없어보이는 단어들을 나열했지만, 사전 학습이 있는 모델에서는 기상청 관련해서는 속보, 기상, 태풍, 등 각종 관청들을 나열하였고, 변화와 관련하여서는 변동, 차이, 변천 등 변화와 비슷한 의미의 단어들을 나열하였다. 이러한 변화가 나타난 이유는 학습한 단어의 수가 사전학습 없던 모델은 3869개인 반면, 사전학습이 있던 모델은 31473개로 약 8배 가량의 단어 수가 사용되었기 때문에 효과가 더 좋은 것으로 판단된다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

PCA는 단어들이 일직선으로 몰려 있는 것을 볼 수 있었고, t_SNE는 원형 모양으로 다소 골고루 퍼져있는 것을 발견할 수 있었다.

In [39]:
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [36]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3869

In [40]:
unique_text[:10]

['리서치', '노스웨스턴대', '보트', '전문가', 'Institute', '하루빨리', 'John', '핵치환', '얼마나', '확률']

In [41]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [43]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:2]
print(pca.shape)
pca

(3869, 2)


array([[-0.63211133, -0.0018809 ],
       [-0.62935041,  0.04893083],
       [ 0.62838855,  0.0051382 ],
       ...,
       [ 0.11357284, -0.08374661],
       [-0.8361438 ,  0.09810282],
       [-0.57391081,  0.00733814]])

In [45]:
# 시각화해보세요!

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [44]:
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:,:2]
print(tsne.shape)
tsne

(3869, 2)


array([[-0.28826058,  1.0320503 ],
       [-0.48801184,  0.79037184],
       [ 0.05094354, -0.13057713],
       ...,
       [ 0.75669277, -0.2632207 ],
       [-0.05949025, -0.4310246 ],
       [-0.608819  ,  0.7751768 ]], dtype=float32)

In [47]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

*  Perplexity를 줄일수록 점점 더 원 모양으로 둥글게 퍼지는 것을 볼 수 있다.

In [50]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 40, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [51]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 30, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [52]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 20, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [49]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [53]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!